In [2]:
# !pip install -U kaleido

In [3]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pd.options.mode.chained_assignment = None

In [4]:
#funções

def tratar_no_info(df):
    df = df.drop(df[(df['Porcentagem'] < 100) & (df['Sorotipo'] == 'S')].index)
    return df


In [5]:
# carregar dados
df_sgtf_original = pd.read_csv('F:\Dados GISAID\dengue\dengueBR_simples_20240416.tsv\dengueBR_simples.tsv',
                               sep='\t')
df_sgtf_original.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO
0,2019-01-04,AC,NaN
1,2019-04-10,AC,NaN
2,2019-12-14,AC,NaN
3,2019-11-19,AC,NaN
4,2019-09-23,AC,NaN


In [6]:
# cópia para df só do BR
df_sgtf_br = df_sgtf_original.copy()

## Brasil

In [7]:
# criar coluna Período (Mes e Ano)
df_sgtf_br['Período'] = df_sgtf_br['DT_NOTIFIC'].str[:7]
df_sgtf_br.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [8]:
df_sgtf_br = df_sgtf_br.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_br['SOROTIPO'] = df_sgtf_br['SOROTIPO'].fillna('Sem informacao')
df_sgtf_br['SG_UF_NOT'] = df_sgtf_br['SG_UF_NOT'].fillna('Sem informacao')



In [9]:
# agrupar por período e sorotipo
df_sgtf_br_agrupado = df_sgtf_br.groupby(['Período', 'SOROTIPO']).count().reset_index()
df_sgtf_br_agrupado = df_sgtf_br_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_br_agrupado.head(10)

,Período,SOROTIPO,SG_UF_NOT
0,2007-01,1,70
1,2007-01,2,19
2,2007-01,3,99
3,2007-01,4,16
4,2007-01,Sem informacao,49339
5,2007-02,1,157
6,2007-02,2,29
7,2007-02,3,142
8,2007-02,4,18
9,2007-02,Sem informacao,85281


In [10]:
# criar coluna ANO (fixar junho)
df_sgtf_br_agrupado['Ano'] = df_sgtf_br_agrupado['Período'].str[:4] + '-06'
df_sgtf_br_agrupado.head(2)

,Período,SOROTIPO,SG_UF_NOT,Ano
0,2007-01,1,70,2007-06
1,2007-01,2,19,2007-06


In [11]:
# renomear colunas
df_sgtf_br_agrupado = df_sgtf_br_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                          'SOROTIPO': 'Sorotipo'})
df_sgtf_br_agrupado.head(5)

,Período,Sorotipo,Casos,Ano
0,2007-01,1,70,2007-06
1,2007-01,2,19,2007-06
2,2007-01,3,99,2007-06
3,2007-01,4,16,2007-06
4,2007-01,Sem informacao,49339,2007-06


In [12]:
# total de casos
df_sgtf_br_agrupado['Casos'].sum()

24031903

### ANO

In [13]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_br_agrupado_ano = df_sgtf_br_agrupado[['Ano', 'Sorotipo', 'Casos']].groupby(['Ano', 'Sorotipo']).sum()
df_sgtf_br_agrupado_ano = df_sgtf_br_agrupado_ano.reset_index()
df_sgtf_br_agrupado_ano

,Ano,Sorotipo,Casos
0,2007-06,1,1797
1,2007-06,2,358
2,2007-06,3,951
3,2007-06,4,215
4,2007-06,Sem informacao,713776
...,...,...,...
80,2024-06,1,39471
81,2024-06,2,19074
82,2024-06,3,300
83,2024-06,4,25


In [14]:
# criar em DF com a porcentagem de sorotipo por caso por ANO
# lista com os ANOS
lista_anos = df_sgtf_br_agrupado_ano['Ano'].unique()
# df vazio
df_sgtf_br_agrupado_ano_freq = pd.DataFrame(columns=['Ano', 'Sorotipo', 'Casos'])

# percorrer cada período
for ano in lista_anos:
    # filtrar o ANO
    df_filtrado_total = df_sgtf_br_agrupado_ano.query('Ano == @ano')
    # filtrar o ANO e sorotipo (retira o que é sem informação) - usado para calacular a extrapolação
    df_filtrado_sorotipo_onlyinfo = df_sgtf_br_agrupado_ano.query('Ano == @ano and Sorotipo != "Sem informacao"')
    
    # filtrar o ANO (mantém o que é sem informação)
    df_filtrado_sorotipo = df_sgtf_br_agrupado_ano.query('Ano == @ano')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo_onlyinfo['Casos'].sum() #so calcula os sorotipos identificados

    # porcentagem de cada sorotipo por ANO pelo total de casos do ANO
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)
    
    #conserta o casos extrapolados para o sem informação repetindo a iformação real em Casos extrapolados
    df_filtrado_sorotipo.loc[df_filtrado_sorotipo['Sorotipo'] == 'Sem informacao', 'CasosExtrap'] = df_filtrado_sorotipo['Casos']

    # porcentagem de cada sorotipo por ANO pelo total de casos do ANO 
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_sgtf_br_agrupado_ano_freq = pd.concat([df_sgtf_br_agrupado_ano_freq, df_filtrado_sorotipo])


df_sgtf_br_agrupado_ano_freq.head(5)


,Ano,Sorotipo,Casos,CasosExtrap,Porcentagem
0,2007-06,1,1797,388023.0,54.11
1,2007-06,2,358,77302.0,10.78
2,2007-06,3,951,205348.0,28.64
3,2007-06,4,215,46425.0,6.47
4,2007-06,Sem informacao,713776,713776.0,99.54


In [15]:
df_sgtf_br_agrupado_ano_freq['Sorotipo'] = df_sgtf_br_agrupado_ano_freq['Sorotipo'].astype('str').str[0]

df_sorotipo1_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "1"').reset_index(drop=True)
df_sorotipo2_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "2"').reset_index(drop=True)
df_sorotipo3_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "3"').reset_index(drop=True)
df_sorotipo4_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "4"').reset_index(drop=True)
df_sorotipoNoInf_ano = df_sgtf_br_agrupado_ano_freq.query('Sorotipo == "S"').reset_index(drop=True)

In [16]:
df_sorotipoNoInf_ano


,Ano,Sorotipo,Casos,CasosExtrap,Porcentagem
0,2007-06,S,713776,713776.0,99.54
1,2009-06,S,598345,598345.0,99.61
2,2010-06,S,1374742,1374742.0,99.53
3,2011-06,S,1145356,1145356.0,99.60
4,2012-06,S,948599,948599.0,99.59
5,2013-06,S,2029417,2029417.0,99.73
6,2014-06,S,959139,959139.0,99.68
7,2015-06,S,2388785,2388785.0,99.76
8,2016-06,S,2299692,2299692.0,99.82
9,2017-06,S,518222,518222.0,99.89


### Período

In [17]:
# criar em DF com a porcentagem de sorotipo por caso por período
# lista com os períodos
lista_periodos = df_sgtf_br_agrupado['Período'].unique()

# df vazio
df_sgtf_br_agrupado_periodo_freq = pd.DataFrame(columns=['Período', 'Sorotipo', 'Casos'])

# percorrer cada período
for periodo in lista_periodos:
    # filtrar o período
    df_filtrado_total = df_sgtf_br_agrupado.query('Período == @periodo')
    # filtrar o período e sorotipo (retira o que é sem informação) - usado para calacular a extrapolação
    df_filtrado_sorotipo_onlyinfo = df_sgtf_br_agrupado.query('Período == @periodo and Sorotipo != "Sem informacao"')
     
    # filtrar o Paríodo (mantém o que é sem informação)
    df_filtrado_sorotipo = df_sgtf_br_agrupado.query('Período == @periodo')


    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo_onlyinfo['Casos'].sum()

    # porcentagem de cada sorotipo por período pelo total de casos do período
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)

    #conserta o casos extrapolados para o sem informação repetindo a iformação real em Casos extrapolados
    df_filtrado_sorotipo.loc[df_filtrado_sorotipo['Sorotipo'] == 'Sem informacao', 'CasosExtrap'] = df_filtrado_sorotipo['Casos']

    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_sgtf_br_agrupado_periodo_freq = pd.concat([df_sgtf_br_agrupado_periodo_freq, df_filtrado_sorotipo])
print(df_sgtf_br_agrupado_periodo_freq.head())

   Período        Sorotipo  Casos      Ano  CasosExtrap  Porcentagem
0  2007-01               1     70  2007-06      17000.0        34.31
1  2007-01               2     19  2007-06       4614.0         9.31
2  2007-01               3     99  2007-06      24043.0        48.53
3  2007-01               4     16  2007-06       3886.0         7.84
4  2007-01  Sem informacao  49339  2007-06      49339.0        99.59


In [18]:
df_sgtf_br_agrupado_periodo_freq['Sorotipo'] = df_sgtf_br_agrupado_periodo_freq['Sorotipo'].astype('str')
df_sgtf_br_agrupado_periodo_freq['Sorotipo'] = df_sgtf_br_agrupado_periodo_freq['Sorotipo'].str[0]

df_sorotipo1_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "1"').reset_index(drop=True)
df_sorotipo2_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "2"').reset_index(drop=True)
df_sorotipo3_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "3"').reset_index(drop=True)
df_sorotipo4_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "4"').reset_index(drop=True)
df_sorotipoNoInf_periodo = df_sgtf_br_agrupado_periodo_freq.query('Sorotipo == "S"').reset_index(drop=True)


In [19]:
#tratar casos de /noinfo quando há informação
tratar_no_info(df_sgtf_br_agrupado_ano_freq)


,Ano,Sorotipo,Casos,CasosExtrap,Porcentagem
0,2007-06,1,1797,388023.0,54.11
1,2007-06,2,358,77302.0,10.78
2,2007-06,3,951,205348.0,28.64
3,2007-06,4,215,46425.0,6.47
5,2009-06,1,1640,425888.0,70.90
...,...,...,...,...,...
78,2023-06,4,17,584.0,0.04
80,2024-06,1,39471,1920009.0,67.05
81,2024-06,2,19074,927827.0,32.40
82,2024-06,3,300,14593.0,0.51


In [20]:
# exportar dfs


df_sgtf_br_agrupado_ano_freq.to_csv('df_dengue_pcr_ano.csv', index=False)
df_sgtf_br_agrupado_periodo_freq.to_csv('df_dengue_pcr_periodo.csv', index=False)

### Gráfico

In [21]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Sorotipo 1',
                     marker_color='mediumvioletred',
                     x=df_sorotipo1_periodo['Período'],
                     y=df_sorotipo1_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 2',
                     marker_color='darkblue',
                     x=df_sorotipo2_periodo['Período'],
                     y=df_sorotipo2_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 3',
                     marker_color='green',
                     x=df_sorotipo3_periodo['Período'],
                     y=df_sorotipo3_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 4',
                     marker_color='gold',
                     x=df_sorotipo4_periodo['Período'],
                     y=df_sorotipo4_periodo['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sem informação',
                     marker_color='gray',
                     x=tratar_no_info(df_sorotipoNoInf_periodo)['Período'],
                     y=tratar_no_info(df_sorotipoNoInf_periodo)['CasosExtrap']),
              secondary_y=False)

# LINHA
fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='mediumvioletred',
                         x=df_sorotipo1_ano['Ano'],
                         y=df_sorotipo1_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='darkblue',
                         x=df_sorotipo2_ano['Ano'],
                         y=df_sorotipo2_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='green',
                         x=df_sorotipo3_ano['Ano'],
                         y=df_sorotipo3_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='gold',
                         x=df_sorotipo4_ano['Ano'],
                         y=df_sorotipo4_ano['Porcentagem']),
              secondary_y=True)

fig.update_layout(barmode='stack',
                  legend_font_size=14,
                  legend_traceorder='normal',
                  legend_orientation='h',
                  legend_xanchor = 'center',
                  legend_yanchor = 'bottom',
                  legend_x= 0.5, legend_y= 1.0,
                  plot_bgcolor='white',
                  xaxis_title="Período de amostragem",
                  yaxis_title="Casos de dengue",)

fig.update_xaxes(showline=True, linewidth=1.5, linecolor='lightgray')
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='lightgray', secondary_y=False)
fig.update_yaxes(secondary_y=True, showline=True, linewidth=1.5, linecolor='lightgray',
                 title_text='Sorotipos (%)', rangemode="tozero")

fig.show()

## Regiões

In [22]:
# cópia para df só de região
df_sgtf_regiao = df_sgtf_original.copy()

In [23]:
# criar coluna Período (Mes e Ano)
df_sgtf_regiao['Período'] = df_sgtf_regiao['DT_NOTIFIC'].str[:7]
df_sgtf_regiao.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [24]:
df_sgtf_regiao = df_sgtf_regiao.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_regiao['SOROTIPO'] = df_sgtf_regiao['SOROTIPO'].fillna('Sem informacao')
df_sgtf_regiao['SG_UF_NOT'] = df_sgtf_regiao['SG_UF_NOT'].fillna('Sem informacao')

In [25]:
# criar Região
dict_regiao = {"AC": "Norte",
               "AM": "Norte",
               "RR": "Norte",
               "RO": "Norte",
               "PA": "Norte",
               "AP": "Norte",
               "TO": "Norte",
               "MA": "Nordeste",
               "PI": "Nordeste",
               "CE": "Nordeste",
               "RN": "Nordeste",
               "PB": "Nordeste",
               "PE": "Nordeste",
               "AL": "Nordeste",
               "SE": "Nordeste",
               "BA": "Nordeste",
               "MT": "Centro-oeste",
               "MS": "Centro-oeste",
               "GO": "Centro-oeste",
               "DF": "Centro-oeste",
               "SP": "Sudeste",
               "RJ": "Sudeste",
               "MG": "Sudeste",
               "ES": "Sudeste",
               "PR": "Sul",
               "SC": "Sul",
               "RS": "Sul",
               "Sem informação": "Sem informacao"}

In [26]:
df_sgtf_regiao['Região'] = df_sgtf_regiao['SG_UF_NOT'].map(dict_regiao)
df_sgtf_regiao.head(1)

,SG_UF_NOT,SOROTIPO,Período,Região
0,AC,Sem informacao,2019-01,Norte


In [27]:
df_sgtf_regiao['Região'].value_counts()

Sudeste         12458170
Nordeste         4073041
Centro-oeste     3661330
Sul              2566460
Norte            1272899
Name: Região, dtype: int64

In [28]:
# agrupar por período, região e sorotipo
df_sgtf_regiao_agrupado = df_sgtf_regiao.groupby(['Região', 'Período', 'SOROTIPO']).count().reset_index()
df_sgtf_regiao_agrupado = df_sgtf_regiao_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_regiao_agrupado.head(2)

,Região,Período,SOROTIPO,SG_UF_NOT
0,Centro-oeste,2007-01,1,17
1,Sul,2007-01,3,6


In [29]:
# criar coluna ANO (fixar junho)
df_sgtf_regiao_agrupado['Ano'] = df_sgtf_regiao_agrupado['Período'].str[:4] + '-06'
df_sgtf_regiao_agrupado.head(2)

,Região,Período,SOROTIPO,SG_UF_NOT,Ano
0,Centro-oeste,2007-01,1,17,2007-06
1,Sul,2007-01,3,6,2007-06


In [30]:
# renomear colunas
df_sgtf_regiao_agrupado = df_sgtf_regiao_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                                  'SOROTIPO': 'Sorotipo'})
df_sgtf_regiao_agrupado.head(10)

,Região,Período,Sorotipo,Casos,Ano
0,Centro-oeste,2007-01,1,17,2007-06
1,Sul,2007-01,3,6,2007-06
2,Sul,2007-01,1,6,2007-06
3,Norte,2007-01,2,16,2007-06
4,Nordeste,2007-01,1,15,2007-06
5,Nordeste,2007-01,2,2,2007-06
6,Nordeste,2007-01,3,10,2007-06
7,Sul,2007-01,4,1,2007-06
8,Nordeste,2007-01,4,3,2007-06
9,Sudeste,2007-01,Sem informacao,12941,2007-06


In [31]:
# total de casos 24031903
df_sgtf_regiao_agrupado['Casos'].sum()

24031900

### ANO

In [32]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_regiao_agrupado_ano = df_sgtf_regiao_agrupado[['Região', 'Ano', 'Sorotipo', 'Casos']].groupby(['Região', 'Ano', 'Sorotipo']).sum()
df_sgtf_regiao_agrupado_ano = df_sgtf_regiao_agrupado_ano.reset_index()
df_sgtf_regiao_agrupado_ano

,Região,Ano,Sorotipo,Casos
0,Centro-oeste,2007-06,1,283
1,Centro-oeste,2007-06,2,5
2,Centro-oeste,2007-06,3,178
3,Centro-oeste,2007-06,4,30
4,Centro-oeste,2007-06,Sem informacao,116953
...,...,...,...,...
391,Sul,2024-06,1,7384
392,Sul,2024-06,2,1576
393,Sul,2024-06,3,105
394,Sul,2024-06,4,5


In [33]:
# função para criar um DF com a porcentagem de sorotipo por caso por ANO e por Região
def calcula_frequencia_sorotipo_ano(df, regiao):
  df_filtrado = df.query('Região == @regiao')
  # lista com os ANOS
  lista_anos = df_filtrado['Ano'].unique()
  # cria df vazio
  df_final = pd.DataFrame(columns=['Região', 'Ano', 'Sorotipo', 'Casos'])

  # percorrer cada período
  for ano in lista_anos:
    # filtrar o ANO
    df_filtrado_total = df_filtrado.query('Ano == @ano')
    # filtrar o ANO e sorotipo
    df_filtrado_sorotipo_onlyinfo = df_filtrado.query('Ano == @ano and Sorotipo != "Sem informação"')

    df_filtrado_sorotipo = df_filtrado.query('Ano == @ano')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo_onlyinfo['Casos'].sum()

    # porcentagem de cada sorotipo por ANO pelo total de casos do ANO
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)

    #conserta o casos extrapolados para o sem informação repetindo a iformação real em Casos extrapolados
    df_filtrado_sorotipo.loc[df_filtrado_sorotipo['Sorotipo'] == 'Sem informação', 'CasosExtrap'] = df_filtrado_sorotipo['Casos']
    
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_final = pd.concat([df_final, df_filtrado_sorotipo])

  df_final['Sorotipo'] = df_final['Sorotipo'].astype('str').str[0]
  df_sorotipo1 = df_final.query('Sorotipo == "1"').reset_index(drop=True)
  df_sorotipo2 = df_final.query('Sorotipo == "2"').reset_index(drop=True)
  df_sorotipo3 = df_final.query('Sorotipo == "3"').reset_index(drop=True)
  df_sorotipo4 = df_final.query('Sorotipo == "4"').reset_index(drop=True)
  df_sorotipoNoInfo = df_final.query('Sorotipo == "S"').reset_index(drop=True)

  return df_sorotipo1, df_sorotipo2, df_sorotipo3, df_sorotipo4, df_sorotipoNoInfo

### Período

In [34]:
# função para criar um DF com a porcentagem de sorotipo por caso por periodo e por Região
def calcula_frequencia_sorotipo_periodo(df, regiao):
  df_filtrado = df.query('Região == @regiao')
  # lista com os ANOS
  lista_periodos = df_filtrado['Período'].unique()
  # cria df vazio
  df_final = pd.DataFrame(columns=['Região', 'Período', 'Sorotipo', 'Casos'])

  # percorrer cada período
  for periodo in lista_periodos:
    # filtrar o periodo
    df_filtrado_total = df_filtrado.query('Período == @periodo')
    # filtrar o período e sorotipo (retira o que é sem informação) - usado para calacular a extrapolação
    
    df_filtrado_sorotipo_onlyinfo = df_sgtf_br_agrupado.query('Período  == @periodo and Sorotipo != "Sem informação"')
     
    # filtrar o Paríodo (mantém o que é sem informação)
    df_filtrado_sorotipo = df_sgtf_br_agrupado.query('Período == @periodo')

    # total de casos do período
    total_casos = df_filtrado_total['Casos'].sum()
    total_casos_sorotipo = df_filtrado_sorotipo_onlyinfo['Casos'].sum()

    # porcentagem de cada sorotipo por período pelo total de casos do período
    df_filtrado_sorotipo['CasosExtrap'] = (df_filtrado_sorotipo['Casos']*total_casos)/total_casos_sorotipo
    df_filtrado_sorotipo['CasosExtrap'] = df_filtrado_sorotipo['CasosExtrap'].round(0)

    #conserta o casos extrapolados para o sem informação repetindo a iformação real em Casos extrapolados
    df_filtrado_sorotipo.loc[df_filtrado_sorotipo['Sorotipo'] == 'Sem informação', 'CasosExtrap'] = df_filtrado_sorotipo['Casos']
    
    df_filtrado_sorotipo['Porcentagem'] = ((df_filtrado_sorotipo['CasosExtrap']/total_casos)*100).round(2)
    # juntar os períodos
    df_final = pd.concat([df_final, df_filtrado_sorotipo])

  df_final['Sorotipo'] = df['Sorotipo'].astype('str').str[0]
  df_sorotipo1 = df_final.query('Sorotipo == "1"').reset_index(drop=True)
  df_sorotipo2 = df_final.query('Sorotipo == "2"').reset_index(drop=True)
  df_sorotipo3 = df_final.query('Sorotipo == "3"').reset_index(drop=True)
  df_sorotipo4 = df_final.query('Sorotipo == "4"').reset_index(drop=True)
  df_sorotipoNoInfo = df_final.query('Sorotipo == "S"').reset_index(drop=True)

  return df_sorotipo1, df_sorotipo2, df_sorotipo3, df_sorotipo4, df_sorotipoNoInfo

In [35]:
# exportar dfs
df_sgtf_regiao_agrupado_ano.to_csv('df_dengue_pcr_regiao_ano.csv', index=False)
df_sgtf_regiao_agrupado.to_csv('df_dengue_pcr_regiao_periodo.csv', index=False)

### Gráfico

In [36]:
# ano
df_sudeste1_ano, df_sudeste2_ano, df_sudeste3_ano, df_sudeste4_ano, df_sudesteNoInfo_ano  = calcula_frequencia_sorotipo_ano(df_sgtf_regiao_agrupado_ano, 'Sudeste')
# periodo
df_sudeste1_per, df_sudeste2_per, df_sudeste3_per, df_sudeste4_per, df_sudesteNoInfo_per= calcula_frequencia_sorotipo_periodo(df_sgtf_regiao_agrupado, 'Sudeste')


In [37]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Sorotipo 1',
                     marker_color='mediumvioletred',
                     x=df_sudeste1_per['Período'],
                     y=df_sudeste1_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 2',
                     marker_color='darkblue',
                     x=df_sudeste2_per['Período'],
                     y=df_sudeste2_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 3',
                     marker_color='green',
                     x=df_sudeste3_per['Período'],
                     y=df_sudeste3_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sorotipo 4',
                     marker_color='gold',
                     x=df_sudeste4_per['Período'],
                     y=df_sudeste4_per['CasosExtrap']),
              secondary_y=False)

fig.add_trace(go.Bar(name='Sem Informação',
                     marker_color='gray',
                     x=tratar_no_info(df_sudesteNoInfo_per)['Período'],
                     y=tratar_no_info(df_sudeste4_per)['CasosExtrap']),
              secondary_y=False)

#LINHA
fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='mediumvioletred',
                         x=df_sudeste1_ano['Ano'],
                         y=df_sudeste1_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='darkblue',
                         x=df_sudeste2_ano['Ano'],
                         y=df_sudeste2_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='green',
                         x=df_sudeste3_ano['Ano'],
                         y=df_sudeste3_ano['Porcentagem']),
              secondary_y=True)

fig.add_trace(go.Scatter(showlegend=False,
                         marker_color='gold',
                         x=df_sudeste4_ano['Ano'],
                         y=df_sudeste4_ano['Porcentagem']),
              secondary_y=True)

fig.update_layout(barmode='stack',
                  legend_font_size=14,
                  legend_traceorder='normal',
                  legend_orientation='h',
                  legend_xanchor = 'center',
                  legend_yanchor = 'bottom',
                  legend_x= 0.5, legend_y= 1.0,
                  plot_bgcolor='white',
                  xaxis_title="Período de amostragem",
                  yaxis_title="Casos de dengue",)

fig.update_xaxes(showline=True, linewidth=1.5, linecolor='lightgray')
fig.update_yaxes(showline=True, linewidth=1.5, linecolor='lightgray', secondary_y=False)
fig.update_yaxes(secondary_y=True, showline=True, linewidth=1.5, linecolor='lightgray',
                 title_text='Sorotipos (%)', rangemode="tozero")

fig.show()

## Estado

In [38]:
# cópia para df só de estado
df_sgtf_uf = df_sgtf_original.copy()

In [39]:
# criar coluna Período (Mes e Ano)
df_sgtf_uf['Período'] = df_sgtf_uf['DT_NOTIFIC'].str[:7]
df_sgtf_uf.head()

,DT_NOTIFIC,SG_UF_NOT,SOROTIPO,Período
0,2019-01-04,AC,NaN,2019-01
1,2019-04-10,AC,NaN,2019-04
2,2019-12-14,AC,NaN,2019-12
3,2019-11-19,AC,NaN,2019-11
4,2019-09-23,AC,NaN,2019-09


In [40]:
df_sgtf_uf = df_sgtf_uf.drop('DT_NOTIFIC', axis=1)

# Substituir os NAN
df_sgtf_uf['SOROTIPO'] = df_sgtf_uf['SOROTIPO'].fillna('Sem informacao')
df_sgtf_uf['SG_UF_NOT'] = df_sgtf_uf['SG_UF_NOT'].fillna('Sem informacao')

In [41]:
dict_uf = {'SP': 'São Paulo',
           'PB': 'Paraíba',
           'SE': 'Sergipe',
           'PI': 'Piauí',
           'PA': 'Pará',
           'MA': 'Maranhão',
           'AC': 'Acre',
           'AP': 'Amapá',
           'CE': 'Ceará',
           'ES': 'Espírito Santo',
           'RS': 'Rio Grande do Sul',
           'MG': 'Minas Gerais',
           'SC': 'Santa Catarina',
           'RR': 'Roraima',
           'AL': 'Alagoas',
           'PE': 'Pernambuco',
           'RN': 'Rio Grande do Norte',
           'MT': 'Mato Grosso',
           'PR': 'Paraná',
           'MS': 'Mato Grosso do Sul',
           'DF': 'Distrito Federal',
           'GO': 'Goiás',
           'BA': 'Bahia',
           'AM': 'Amazonas',
           'RO': 'Rondônia',
           'RJ': 'Rio de Janeiro',
           'TO': 'Tocantins'}

In [42]:
df_sgtf_uf['Estado'] = df_sgtf_uf['SG_UF_NOT'].map(dict_uf)
print(df_sgtf_uf.head(1))
print(df_sgtf_uf['SG_UF_NOT'].unique())


  SG_UF_NOT        SOROTIPO  Período Estado
0        AC  Sem informacao  2019-01   Acre
['AC' 'AM' 'RN' 'GO' 'DF' 'RJ' 'SP' 'PR' 'MS' 'AL' 'PE' 'MG' 'MT' 'SE'
 'ES' 'BA' 'CE' 'PB' 'RO' 'PA' 'RR' 'SC' 'AP' 'RS' 'TO' 'PI' 'MA'
 'Sem informacao']


In [43]:
df_sgtf_uf['Estado'].value_counts()

São Paulo              6124274
Minas Gerais           4409063
Paraná                 1882000
Goiás                  1841232
Rio de Janeiro         1412548
Bahia                  1063748
Ceará                   880219
Pernambuco              784227
Mato Grosso do Sul      722286
Distrito Federal        588540
Espírito Santo          512285
Mato Grosso             509272
Santa Catarina          454179
Rio Grande do Norte     386148
Alagoas                 299236
Acre                    275475
Tocantins               273353
Paraíba                 263670
Pará                    248124
Rio Grande do Sul       230281
Amazonas                219234
Piauí                   169232
Rondônia                161520
Maranhão                136723
Sergipe                  89838
Roraima                  52131
Amapá                    43062
Name: Estado, dtype: int64

In [44]:
# agrupar por período, uf e sorotipo
df_sgtf_uf_agrupado = df_sgtf_uf.groupby(['Estado', 'Período', 'SOROTIPO']).count().reset_index()
df_sgtf_uf_agrupado = df_sgtf_uf_agrupado.sort_values('Período').reset_index(drop=True)
df_sgtf_uf_agrupado.head(2)

,Estado,Período,SOROTIPO,SG_UF_NOT
0,Acre,2007-01,Sem informacao,268
1,Distrito Federal,2007-01,Sem informacao,166


In [45]:
# criar coluna ANO (fixar junho)
df_sgtf_uf_agrupado['Ano'] = df_sgtf_uf_agrupado['Período'].str[:4] + '-06'
df_sgtf_uf_agrupado.head(2)

,Estado,Período,SOROTIPO,SG_UF_NOT,Ano
0,Acre,2007-01,Sem informacao,268,2007-06
1,Distrito Federal,2007-01,Sem informacao,166,2007-06


In [46]:
# renomear colunas
df_sgtf_uf_agrupado = df_sgtf_uf_agrupado.rename(columns={'SG_UF_NOT': 'Casos',
                                                          'SOROTIPO': 'Sorotipo'})
df_sgtf_uf_agrupado.head(10)

,Estado,Período,Sorotipo,Casos,Ano
0,Acre,2007-01,Sem informacao,268,2007-06
1,Distrito Federal,2007-01,Sem informacao,166,2007-06
2,Distrito Federal,2007-01,1,4,2007-06
3,Rio Grande do Sul,2007-01,Sem informacao,35,2007-06
4,Rio de Janeiro,2007-01,3,16,2007-06
5,Rio de Janeiro,2007-01,4,1,2007-06
6,Rio de Janeiro,2007-01,Sem informacao,2460,2007-06
7,Ceará,2007-01,Sem informacao,2187,2007-06
8,Ceará,2007-01,4,2,2007-06
9,Ceará,2007-01,1,4,2007-06


In [47]:
# total de casos 22901570
df_sgtf_uf_agrupado['Casos'].sum()

24031900

### ANO

In [48]:
# criar um df agrupado apenas pelo ANO para as linhas de frequência anuais de cada sorotipo
df_sgtf_uf_agrupado_ano = df_sgtf_uf_agrupado[['Estado', 'Ano', 'Sorotipo', 'Casos']].groupby(['Estado', 'Ano', 'Sorotipo']).sum()
df_sgtf_uf_agrupado_ano = df_sgtf_uf_agrupado_ano.reset_index()
df_sgtf_uf_agrupado_ano

,Estado,Ano,Sorotipo,Casos
0,Acre,2007-06,2,1
1,Acre,2007-06,3,1
2,Acre,2007-06,4,1
3,Acre,2007-06,Sem informacao,2355
4,Acre,2009-06,1,10
...,...,...,...,...
1616,Tocantins,2023-06,2,26
1617,Tocantins,2023-06,Sem informacao,3222
1618,Tocantins,2024-06,1,37
1619,Tocantins,2024-06,2,58


In [49]:
# exportar dfs
df_sgtf_uf_agrupado_ano.to_csv('df_dengue_pcr_estado_ano.csv', index=False)
df_sgtf_uf_agrupado.to_csv('df_dengue_pcr_estado_periodo.csv', index=False)
df_sgtf_uf_agrupado

,Estado,Período,Sorotipo,Casos,Ano
0,Acre,2007-01,Sem informacao,268,2007-06
1,Distrito Federal,2007-01,Sem informacao,166,2007-06
2,Distrito Federal,2007-01,1,4,2007-06
3,Rio Grande do Sul,2007-01,Sem informacao,35,2007-06
4,Rio de Janeiro,2007-01,3,16,2007-06
...,...,...,...,...,...
11228,Bahia,2024-04,Sem informacao,7444,2024-06
11229,Roraima,2024-04,Sem informacao,31,2024-06
11230,Mato Grosso do Sul,2024-04,Sem informacao,829,2024-06
11231,Mato Grosso,2024-04,1,1,2024-06
